# Collecting various gridded datasets to use as predictors in a NEE model

- ANU climate data
- MODIS LAI
- Soil moisture from GRAFS

Results are saved to `/g/data/os22/chad_tmp/NEE_modelling/results/input_data/input_data_<YYYY>.nc`

In [ ]:
# !pip install xarray
# !pip install datacube
# !pip install --extra-index-url="https://packages.dea.ga.gov.au" \
#   odc-algo

# !pip install rioxarray
# !pip install odc-geo
# !pip install dea_tools
# !pip install joblib
# !pip install tqdm
# !pip install geopandas
# !pip install -U scikit-learn
# !pip install dask-ml

In [1]:
from datacube.utils.dask import start_local_dask
import sys
sys.path.append('/g/data/os22/chad_tmp/NEE_modelling/')
from collect_gridded_data import collect_gridded_data

In [2]:
client = start_local_dask(mem_safety_margin='2Gb')
print(client)

<Client: 'tcp://127.0.0.1:35511' processes=1 threads=16, memory=44.92 GiB>


## Analysis Parameters

In [3]:
time=('2019')

In [4]:
%%time
data = collect_gridded_data(time=time, chunks=dict(lat=1000, lon=1000))

   Extracting ANU Climate data
   Extracting MODIS LAI
   Extracting soil moisture
   Reprojecting datasets
   Adding a lag
   Exporting netcdf
CPU times: user 41 s, sys: 14.3 s, total: 55.3 s
Wall time: 6min 36s


### Try to stich together SIF

In [ ]:
# import xarray as xr
# xr.open_dataarray('/g/data/os22/chad_tmp/NEE_modelling/data/SIF/GOSIF/GOSIF_2000.M03.tif')